In [1]:
league = "EPL"
season = "2024"

In [2]:
import pandas as pd
import requests
import re
from unidecode import unidecode

In [3]:
import understatapi
client = understatapi.UnderstatClient()

In [4]:
from sqlalchemy import create_engine
import psycopg2

# Database connection settings (replace with your credentials)
db_config = {
    'dbname': 'understat_shots_db',
    'user': 'ichadha',
    'password': 'ichadhapg',
    'host': 'localhost',
    'port': 5432,
}

# Create a connection engine
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}")

def save_to_postgres(dataframe, table_name):
    try:
        # Append data to the PostgreSQL table
        dataframe.to_sql(table_name, engine, if_exists='append', index=False)
        print(f"Data successfully saved to {table_name} table.")
    except Exception as e:
        print(f"Error saving data to PostgreSQL: {e}")


In [5]:
## EPL , Bundesliga , Serie_A , Ligue_1 , La_Liga
#league = "EPL"
#season = "2024"

league_data = client.league(league=league).get_match_data(season=season)

In [6]:
matches = []

for match in league_data:
    match_data = {
        'id': match['id'],
        'home_team': match['h']['title'],
        'away_team': match['a']['title'],
        'home_goals': match['goals']['h'],
        'away_goals': match['goals']['a'],
        'home_xG': match['xG']['h'],
        'away_xG': match['xG']['a'],
        'datetime': match['datetime']
        #'forecast': match['forecast']
    }
    matches.append(match_data)

# Convert the list of dictionaries into a DataFrame
matches_df = pd.DataFrame(matches)

In [7]:
all_shot_data = []

import datetime

current_timestamp = datetime.datetime.now().timestamp()

for index,row in matches_df.iterrows():
    match_datetime = pd.to_datetime(row['datetime']).timestamp()
    
    if match_datetime <= current_timestamp:
        #print(row['id'])
        try:
            shot_data = client.match(match=row['id']).get_shot_data()
        except Exception as e:
            print(e)
        all_shot_data.append(shot_data)

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [8]:
# Initialize an empty list to hold individual shot records
compiled_shot_data = []

# Loop through each match in `all_shot_data`
for match in all_shot_data:
    # Get home shots and away shots from the match
    home_shots = match.get('h', [])
    away_shots = match.get('a', [])
    
    # Add a column to indicate if it's a home or away shot, then extend our list
    for shot in home_shots:
        shot['h_a'] = 'h'  # Indicate as home shot
        compiled_shot_data.append(shot)
    
    for shot in away_shots:
        shot['h_a'] = 'a'  # Indicate as away shot
        compiled_shot_data.append(shot)

# Convert the list of shot records to a DataFrame
compiled_shot_df = pd.DataFrame(compiled_shot_data)

In [9]:
compiled_shot_df['league'] = league

In [10]:
#compiled_shot_df.to_csv(f'C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/Data/{league}/2024-25/shot_data.csv')
save_to_postgres(compiled_shot_df, 'understat_shots_tb')

Error saving data to PostgreSQL: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "understat_shots_tb_pkey"
DETAIL:  Key (id)=(584630) already exists.

[SQL: INSERT INTO understat_shots_tb (id, minute, result, "X", "Y", "xG", player, h_a, player_id, situation, season, "shotType", match_id, h_team, a_team, h_goals, a_goals, date, player_assisted, "lastAction", league) VALUES (%(id)s, %(minute)s, %(result)s, %(X)s, %(Y)s, %(xG)s, %(player)s, %(h_a)s, %(player_id)s, %(situation)s, %(season)s, %(shotType)s, %(match_id)s, %(h_team)s, %(a_team)s, %(h_goals)s, %(a_goals)s, %(date)s, %(player_assisted)s, %(lastAction)s, %(league)s)]
[parameters: ({'id': '584630', 'minute': '19', 'result': 'MissedShots', 'X': '0.970999984741211', 'Y': '0.6730000305175782', 'xG': '0.050078392028808594', 'player': 'Bruno Fernandes', 'h_a': 'h', 'player_id': '1228', 'situation': 'OpenPlay', 'season': '2024', 'shotType': 'LeftFoot', 'match_id': '26602', 'h_team': 'Manchester United',